In [6]:
from omg_dosimetry.tiff2dose import Gaf
from omg_dosimetry.calibration import load_lut_array
from pathlib import Path
from image import load

### Is LUT from Dosepy compatible with OMG Gaf class?

In [7]:
verif_path = Path("/home/luis/Descargas") / "VERIFICACION_1.tif" # Personal laptop
verif = load(verif_path)

In [8]:
verif

In [13]:
from omg_dosimetry.calibration import save_lut
import numpy as np

# Load lut (./user/test_lut.npy)
np.load("../user/calibration/test_lut.npy")
#save_lut("/home/luis/Descargas")

array([[    0.,    50.,   100.,   150.,   200.,   300.,   500.,   800.,
         1000.],
       [    0.,    50.,   100.,   150.,   200.,   300.,   500.,   800.,
         1000.],
       [37584., 30194., 26092., 23218., 21039., 18259., 14913., 12579.,
        11700.],
       [43683., 32512., 27007., 23535., 21151., 18356., 15515., 13682.,
        12980.],
       [42031., 33710., 28796., 25130., 22258., 18509., 13893., 10956.,
         9956.],
       [27037., 24361., 22472., 20990., 19708., 17912., 15330., 13098.,
        12165.]])

In [ ]:
gaf1 = Gaf(
    path=demo_path,
    lut_file=lut_file,
    fit_type='rational',
    clip=1100,
    )

#### No es compatible.
Habría que modificar la clase Gaf para que lea el archivo LUT desde un array, así como modificar la función load.

#### ¿Cómo guardar una clase Dosepy.calibration.Calibration?
https://realpython.com/python-pickle-module/